In [1]:
## Importamos librerias de webscrapping
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
## url para traer los datos de la premier league
#standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
standings_url = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"

In [5]:
## asignamos la url a request.get
data = requests.get(standings_url)
# Crea un objeto BeautifulSoup para analizar el contenido HTML de 'data'.
soup = BeautifulSoup(data.text)

In [6]:
# Selecciona la primera tabla con la clase 'stats_table' en el HTML analizado.
standings_table = soup.select('table.stats_table')[0]

# Busca todos los enlaces ('a') dentro de la tabla seleccionada.
links = standings_table.find_all('a')

# Extrae los atributos 'href' de los enlaces y los almacena en una lista.
links = [l.get("href") for l in links]

# Filtra los enlaces para mantener solo aquellos que contienen '/squads/' en su URL.
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

## Listado de equipos premier League

In [ ]:
team_urls

['https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/2022-2023/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/2022-2023/Manchester-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/2022-2023/Newcastle-United-Stats',
 'https://fbref.com/en/squads/822bd0ba/2022-2023/Liverpool-Stats',
 'https://fbref.com/en/squads/d07537b9/2022-2023/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8602292d/2022-2023/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/2022-2023/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/2022-2023/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/2022-2023/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/2022-2023/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2022-2023/Chelsea-Stats',
 'https://fbref.com/en/squads/8cec06e1/2022-2023/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/7c21e445/2022-2023/West-Ham-United-Stats'

## Funcion para automatizar la busqueda de estadisticas

In [ ]:
def automatization (base_url):
  years_to_replace = ["2022-2023", "2021-2022"]
  #years_to_replace = ["2022-2023", "2021-2022", "2020-2021", "2019-2020", "2018-2019", "2017-2018", "2016-2017", "2015-2016"]
  df1 = pd.DataFrame()  # Inicializa un DataFrame vacío
  for old_year in years_to_replace:
      # Crear la URL con la fecha actual
      url = base_url.format(old_year)
      data1 = requests.get(url)
      matches = pd.read_html(data1.text, match="Scores & Fixtures")[0]
      columns_to_remove = ['Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 'Notes']
      matches = matches.drop(columns=columns_to_remove)

      # Crea un objeto BeautifulSoup para analizar el contenido HTML contenido en 'data.text'.
      soup = BeautifulSoup(data1.text)

      # Encuentra todos los elementos de anclaje ('a') en el HTML analizado y los almacena en una lista.
      link = soup.find_all('a')

      # Extrae los valores del atributo 'href' de los elementos de anclaje y los almacena en una lista.
      link = [l.get("href") for l in link]

      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/shooting/' en su URL.
      links_shooting = [l for l in link if l and 'all_comps/shooting/' in l]
      data2 = requests.get(f"https://fbref.com{links_shooting[0]}")
      shooting = pd.read_html(data2.text, match="Shooting")[0]
      shooting.columns = shooting.columns.droplevel()
      columns_to_remove = ['Gls','SoT%','G/Sh',	'G/SoT', 'Dist', 'FK',
       'PK', 'PKatt','xG','npxG', 'npxG/Sh', 'np:G-xG',
       'Match Report' ]
      shooting = shooting.drop(columns=columns_to_remove)

      ## realizo el merged entre shooting y matches
      df = matches.merge(shooting, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent'])

      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/keeper/' en su URL.
      links_keeper = [l for l in link if l and 'all_comps/keeper/' in l]
      data2 = requests.get(f"https://fbref.com{links_keeper[0]}")
      goalkeeping = pd.read_html(data2.text, match="Goalkeeping")[0]
      goalkeeping.columns = goalkeeping.columns.droplevel()
      goalkeeping=goalkeeping[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent','SoTA',
                         'Save%', 'CS', 'PSxG', 'PKatt','PKsv', 'PKm']]

      ## realizo el merged entre shooting y matches
      df = df.merge(goalkeeping, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'])

      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/passing/' en su URL.
      links_passing = [l for l in link if l and 'all_comps/passing/' in l]
      data2 = requests.get(f"https://fbref.com{links_passing[0]}")
      passing = pd.read_html(data2.text, match="Passing")[0]
      passing.columns = passing.columns.droplevel()
      passing=passing[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent',
                 "TotDist", "PrgDist", "xA", "KP", "1/3", "PPA", "CrsPA", "PrgP"]]
      df = df.merge(passing, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'])

      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/passing_types/' en su URL.
      links_passing_types = [l for l in link if l and 'all_comps/passing_types/' in l]
      data2 = requests.get(f"https://fbref.com{links_passing_types[0]}")
      passing_types = pd.read_html(data2.text, match="Pass Types")[0]
      passing_types.columns = passing_types.columns.droplevel()
      passing_types =passing_types [['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue',
                                     'Result', 'Opponent', 'Att', 'Live', 'Dead', 'FK', 'CK', 'TB',
                                     'Crs', 'TI', 'Off', 'Blocks']]
      df = df.merge(passing_types, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'])


      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/gca/' en su URL.
      links_gca = [l for l in link if l and 'all_comps/gca/' in l]
      data2 = requests.get(f"https://fbref.com{links_gca[0]}")
      Goal_Shot = pd.read_html(data2.text, match="Goal and Shot Creation")[0]
      Goal_Shot.columns = Goal_Shot.columns.droplevel()
      Goal_Shot = Goal_Shot[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent',
                       'SCA', 'GCA','TO']]
      df = df.merge(Goal_Shot, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'])

      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/defense/' en su URL.
      links_defense = [l for l in link if l and 'all_comps/defense/' in l]
      data2 = requests.get(f"https://fbref.com{links_defense[0]}")
      Defensive = pd.read_html(data2.text, match="Defensive Actions")[0]
      Defensive.columns = Defensive.columns.droplevel()
      Defensive=Defensive[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent','TklW', 'Tkl%','Blocks', 'Sh', 'Pass','Int']]
      df = df.merge(Defensive, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'])

      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/possession/' en su URL.
      links_possession = [l for l in link if l and 'all_comps/possession/' in l]
      data2 = requests.get(f"https://fbref.com{links_possession[0]}")
      Possession = pd.read_html(data2.text, match="Possession")[0]
      Possession.columns = Possession.columns.droplevel()
      Possession=Possession[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent','Poss', 'Touches', 'Def Pen',
      'Mid 3rd', 'Att 3rd', 'Succ']]
      df = df.merge(Possession, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'])
      # Filtra la lista de enlaces para conservar solo aquellos que contienen la cadena '/all_comps/misc/' en su URL.
      links_misc = [l for l in link if l and 'all_comps/misc/' in l]
      data2 = requests.get(f"https://fbref.com{links_misc[0]}")
      Miscellaneous = pd.read_html(data2.text, match="Miscellaneous Stats")[0]
      Miscellaneous.columns = Miscellaneous.columns.droplevel()
      Miscellaneous=Miscellaneous[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent',
                                   "CrdY", "CrdR", "Fld", "PKwon", "PKcon", "Recov"]]
      df = df.merge(Miscellaneous, on=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'])

      # Concatenar dataframes para cada for
      df1 = pd.concat([df1, df], ignore_index=True)
  return (df1)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
base_url = "https://fbref.com/en/squads/b8fd03ef/{}/Manchester-City-Stats"
df = automatization(base_url)

In [ ]:
resulting_dataframe.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss_x,Sh_x,SoT,G-xG,SoTA,Save%,CS,PSxG,PKatt,PKsv,PKm,TotDist,PrgDist,xA,KP,1/3,PPA,CrsPA,PrgP,Att,Live,Dead,FK,CK,TB,Crs,TI,Off,Blocks_x,SCA,GCA,TO,TO,TklW,Tkl%,Blocks_y,Sh_y,Pass,Int,Poss_y,Touches,Def Pen,Mid 3rd,Att 3rd,Succ,CrdY,CrdR,Fld,PKwon,PKcon,Recov
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1,3,Liverpool,NaN,NaN,57,14,8,NaN,3,33.3,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,NaN,NaN,14,57.0,NaN,NaN,NaN,NaN,NaN,1,0,12,NaN,NaN,NaN
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,2.2,0.5,75,13,1,-0.2,1,100.0,1.0,0.1,0,0,0,12789.0,3979.0,1.6,11.0,52.0,9.0,0.0,51.0,869.0,836.0,32.0,10.0,4.0,1.0,18,12.0,1.0,4.0,27.0,4.0,0.0,0.0,4,85.7,7.0,3.0,4.0,3,75.0,936.0,55.0,484.0,234.0,14.0,1,0,8,1.0,0.0,47.0
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,1.7,0.1,67,19,7,1.3,1,100.0,1.0,0.1,0,0,0,11744.0,3043.0,1.8,17.0,76.0,13.0,3.0,73.0,739.0,703.0,34.0,10.0,11.0,3.0,27,11.0,2.0,3.0,36.0,6.0,3.0,0.0,4,40.0,11.0,1.0,10.0,5,67.0,814.0,20.0,409.0,325.0,10.0,0,0,9,0.0,0.0,41.0
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,2.1,1.8,69,21,10,0.9,5,40.0,0.0,2.3,0,0,0,9782.0,3100.0,2.0,17.0,48.0,9.0,2.0,60.0,660.0,612.0,47.0,15.0,5.0,7.0,20,18.0,1.0,10.0,40.0,6.0,2.0,0.0,5,30.8,6.0,3.0,3.0,5,69.0,756.0,66.0,364.0,202.0,13.0,2,0,14,0.0,0.0,47.0
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,2.2,0.1,74,18,5,1.8,2,0.0,0.0,0.3,0,0,0,12510.0,3364.0,1.8,15.0,82.0,22.0,8.0,81.0,822.0,782.0,38.0,9.0,6.0,1.0,29,18.0,2.0,3.0,31.0,7.0,2.0,0.0,10,50.0,7.0,0.0,7.0,5,74.0,894.0,28.0,422.0,364.0,10.0,1,0,8,0.0,0.0,47.0


In [ ]:
resulting_dataframe.columns.values[43] = 'TO_SCA'

In [ ]:
resulting_dataframe['TO_SCA']

0      NaN
1      0.0
2      3.0
3      2.0
4      2.0
      ... 
114    1.0
115    2.0
116    0.0
117    4.0
118    4.0
Name: TO_SCA, Length: 119, dtype: float64

In [ ]:
duplicados = resulting_dataframe.columns[resulting_dataframe.columns.duplicated()].tolist()
duplicados

[]